In [43]:
# imports here
import numpy as np
import matplotlib.pyplot as plt
import time
from copy import deepcopy


In [44]:
# Test Function
def sphere_function(x):
    ##return sum((x**2 - 2*x - 4) ** 2)
    return sum(x**2)

In [45]:
# Problem to be defined here as a class (or struct)

class problem:
    
    def __init__(self):
    # Cost function
        self.cost_function = sphere_function
    # number of genes (variables) in an individial
        self.number_of_genes = 5

    #  Max gene value
        self.maximum_value = 10
    # Min Gene Value
        self.minimum_value = -10


In [46]:
#Genetic Algorithm Parameters here as a class or struct
class parameters():
    def __init__(self):
        
# Number of iterations of genetic algorithm
        self.number_of_iterations = 60
# Number of individuals in Population 
        self.number_in_population = 15
    
# Rate of new children pre iteration
        self.child_rate = 0.5
# Number of children 
        self.number_of_children = self.number_in_population * self.child_rate
# Exploration Rate (Gamma)
        self.explore_rate = 0.2
    
# Mutation Rate (Probability that a given gene will be mutated)
        self.mutation_rate = 0.1
# Mutation Range (Determines the amount of change of an individual gene)
        self.mutation_range = 0.1
# Percentage Change of Explore Rate after each iteration
        self.explore_rate_change = 0.98
# Percentage Change of Mutation Rate after each iteration
        self.mutation_rate_change = 0.98


    def to_String(self):
        return ("Population: {}  Child Rate: {}  Explore Rate: {}  Mutation Rate: {}  Mutation Range: {} Explore Rate Change: {}"
                .format(self.number_in_population, self.child_rate, self.explore_rate, self.mutation_rate, self.mutation_range, self.explore_rate_change))



In [47]:
# Structure for an individual of the population as a class or struct
class individual:
    

# Genotype or Chromosone
    chromosone = None
# Cost for the individual
    cost = 0
# Fitness of the individual (for selection of parents)
    fitness = 0
# Constructor(s)

    # if problem is passed as parameter to constructor, a random individual is generated
    #   https://docs.python.org/3/glossary.html#term-argument
    def __init__(self, problem = None):
        
        if problem is not None:
            self.chromosone = np.random.uniform(problem.minimum_value,problem.maximum_value,problem.number_of_genes)
            self.cost = problem.cost_function(self.chromosone)
            if self.cost is not 0:
                self.fitness = (float)(1.0)/self.cost
            else:
                #For this test 0.000001 is the minimum cost for an individual, so if cost is 0, it should be less than that
                self.fitness = (float)(1.0)/0.0000009
# Crossover with another individual  method
    def crossover(self, other_individual, explore_rate):
        child1 = deepcopy(self)
        child2 = deepcopy(other_individual)
        alpha = np.random.uniform(-explore_rate, 1+explore_rate, child1.chromosone.shape)
        child1.chromosone = alpha*self.chromosone + (1-alpha) * other_individual.chromosone
        child2.chromosone = alpha*other_individual.chromosone + (1-alpha) * self.chromosone
        return child1, child2
        
# Mutate method
    def mutate(self, rate, range_of_change):
        for i in range(len(self.chromosone)):
            if np.random.rand() < rate:
                self.chromosone[i] += np.random.randn() * range_of_change


In [48]:
def roulette_parent_selection(population):
    wheel_size = 0
    for i in range(len(population)):
        wheel_size += population[i].fitness
    
    index1 = np.random.uniform(0, wheel_size)
    index2 = np.random.uniform(0, wheel_size)
    if index1 == index2:
        return roulette_parent_selection(population)
    else:
        selection1 = 0
        selection2 = 0
        parent1 = individual()
        parent2 = individual()
        for i in range(len(population)):
            selection1 += population[i].fitness
            selection2 += population[i].fitness
            if selection1 >= index1:
                parent1 = population[i]
            elif selection2 >= index2:
                parent2 = population[i]
    
        if parent1 == parent2 or parent1.chromosone is None or parent2.chromosone is None:
            return roulette_parent_selection(population)
        else:
            return parent1, parent2

In [49]:
def choose_distinct_pair_from(population_size):
    index1 = np.random.randint(population_size)
    index2 = np.random.randint(population_size)
    if index1 == index2:
        return choose_distinct_pair_from(population_size)
    else:
        return index1, index2

In [64]:
# Main Genetic method

def run_genetic (problem, parameters, max_error = 0):
    
    #  read problem
    cost_function = problem.cost_function
    
    # read parameters
    number_in_population = parameters.number_in_population
    number_of_iterations = parameters.number_of_iterations
    number_of_children = parameters.number_of_children
    explore_rate = parameters.explore_rate
    mutation_rate = parameters.mutation_rate
    mutation_range = parameters.mutation_range
    explore_rate_change = parameters.explore_rate_change
    mutation_rate_change = parameters.mutation_rate_change
    
    
    # placeholder for best solution
    best_solution = deepcopy(individual())
    best_solution.cost = np.infty
    
    # placeholder for best cost at each iteration
    #best_costs = []
    
    population = []
    #  initialize population for the above problem, i.e. generate random individuals 
    for i in range(number_in_population):
        new_individual = individual(problem)
        population.append(new_individual)
        if new_individual.cost < best_solution.cost:
            best_solution = deepcopy(new_individual)
        
    
     # Main Loop for algorithm   (number of iterations)
    
    for iteration in range(number_of_iterations):
        
        # generate a new population of children
        children = []
        
        # How many children? 
        while len(children) < number_of_children:
                
                # Select 2 Parents
                #Roulette Selection
                #parent1, parent2 = roulette_parent_selection(population)
                
                #Initial Selection
                parent1_index, parent2_index = choose_distinct_pair_from(number_in_population)
                parent1 = population[parent1_index]
                parent2 = population[parent2_index]
                
                # Use crossover to produce 2 children
                child1, child2 = parent1.crossover(parent2, explore_rate)
                
                #explore_rate *= explore_rate_change
                
                # Mutate these children
                child1.mutate(mutation_rate, mutation_range)
                child2.mutate(mutation_rate, mutation_range)
                
                mutation_rate *= mutation_rate_change
                
                # calculate costs for these children
                child1.cost = cost_function(child1.chromosone)
                child2.cost = cost_function(child2.chromosone)
                
                # add to the children population of new children
                children.append(child1)
                children.append(child2)
                
        # Merge parent and child population
        population += children
        
        # Sort into ascending order for cost
        population = sorted(population, key = lambda x: x.cost)
        
        # Select population next iteration
        population = population[0:number_in_population]
        
        # Update best solution
        if population[0].cost < best_solution.cost:
            best_solution = deepcopy(population[0])
        
        # print iteration results
        #print("Iteration {} Cost {}".format(iteration, best_solution.cost))
        
        #output results ?? class/struct/ best solution/ population?
        #best_costs.append(best_solution.cost)
        
        if best_solution.cost < max_error:
            #print("Stopped at iteration: {}".format(iteration))
            break
    
    #print("Ended at iteration: {}".format(iteration))
                
    #plt.plot(best_costs)
    #plt.semilogy(best_costs)
    #plt.xlabel("Iteration")
    #plt.ylabel("Best Cost")
    #plt.title("Genetic Algorithm \n {}".format(parameters.to_String()))
    return population, best_solution
        
    
    
    
    
    
    

In [51]:
prob = problem()

In [52]:
pars = parameters()

In [53]:
final_pop, best_solution = run_genetic(prob, pars)

In [54]:
def test_genetic(number_of_runs, problem, parameters, max_error):
    start_time = time.perf_counter()
    for i in range(number_of_runs):
        run_genetic(problem, parameters, max_error)
    return(time.perf_counter() - start_time)

In [65]:
final_test = test_genetic(100, prob, pars, 0.00001)

In [56]:
pars.to_String()

'Population: 15  Child Rate: 0.5  Explore Rate: 0.2  Mutation Rate: 0.1  Mutation Range: 0.1 Explore Rate Change: 0.98'

# Using initial parent selection, no explore rate or mutation changes

In [57]:
print(final_test)

1.3242572999988624


# Using roulette parent selection

In [60]:
print(final_test)

1.879272199999832


# Using explore rate change

In [63]:
print(final_test)

1.4097194999994827


# Using mutation rate change

In [66]:
print(final_test)

1.2181888000013714


# Conclusion, changing the mutation rate after each iteration is the only method shown that improves the time in which it takes to get a result